In [1]:
"""
Master Notebook 

"""

'\nMaster Notebook \n\n'

In [22]:
"""Imports
"""
import pandas as pd 
from pathlib import Path 
from dotenv import load_dotenv 
import os
import matplotlib.pyplot as plt
import hvplot.pandas
import numpy as np
import hvplot.pandas
import alpaca_trade_api as tradeapi
import panel as pn
import matplotlib 
from datetime import datetime
import requests

from scipy import stats
from tabulate import tabulate
# import pandas_datareader as web

ModuleNotFoundError: No module named 'pandas_datareader'

In [27]:

#Loading in my environments: dotenv an 

load_dotenv("api.env")
APCA_API_BASE_URL = "https://paper-api.alpaca.markets" 


#Setting up my Alpaca API Keys and creating the Alpaca API object: 

apikey = os.getenv("ALPACA_API_KEY")
secret = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    apikey, secret,
    api_version = "v2"
)



#Calling in my securities and setting up a mock portfolio 

assets = ["ARKK","SPY"]


#Getting my ticker information from Alpaca API 

timeframe = "1Day"
start_date = pd.Timestamp("2018-12-31", tz = "America/New_York").isoformat()
end_date = pd.Timestamp("2023-12-29", tz = "America/New_York").isoformat()

#Getting my ticker information for the funds: ARKK and SPY 

df_assets = api.get_bars(
    assets,
    timeframe,
    start = start_date,
    end = end_date
).df 

# df_assets.head()

#Dropped column head: Symbol and made SPY & ARKK the symbol head 
SPY = df_assets[df_assets['symbol'] == 'SPY'].drop('symbol', axis = 1)
ARKK = df_assets[df_assets['symbol'] == 'ARKK'].drop('symbol', axis = 1)

#Concat the dataframe to show SPY & ARKK as the column head. 
df_assets_concat = pd.concat([SPY,ARKK], axis = 1, keys = ["SPY","ARKK"])
# df_assets_concat

#Setting up a blank dataframe to input my ARKK & SPY 
df_ARKK_SPY = pd.DataFrame() 

#Isolated the SPY and ARKK close dates 
df_ARKK_SPY["SPY"] = df_assets_concat["SPY"]["close"]
df_ARKK_SPY["ARKK"] = df_assets_concat["ARKK"]["close"]

#Dropped the time index from the date 
df_ARKK_SPY.index = df_ARKK_SPY.index.date
# df_ARKK_SPY.head()

#Calculated the daily returns of ARKK & SPY 
df_ARKK_SPY_daily = df_ARKK_SPY.pct_change().dropna()
# df_ARKK_SPY_daily.head()

#Plotting the daily returns 
df_ARKK_SPY_daily.hvplot.line(title = "ARKK & SPY Daily Returns")

######


df_ARKK_SPY.head()

,SPY,ARKK
2018-12-31,250.08,37.19
2019-01-02,250.23,37.42
2019-01-03,244.15,36.19
2019-01-04,252.39,38.43
2019-01-07,254.29,40.10


In [25]:
#Traynor Ratio 
rfr = 5.30 
def traynor_ratio(df_ARKK_SPY,start_date, end_date,rfr):
    data = pd.DataFrame()
    ret = pd.DataFrame()
    bench = pd.DataFrame()
    for close in df_ARKK_SPY["ARKK"]:
    

In [24]:
"""Aditya's Section - Data
"""


def get_historical_bitcoin_data(start_date, end_date):
    # Convert start and end dates to Unix timestamp
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
    end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
    # Calculate the number of days between start and end dates
    days_difference = (end_timestamp - start_timestamp) // (24 * 3600) # seconds in a day
    # API endpoint for historical prices
    api_url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart'
    # Define parameters for the API request
    params = {
        'vs_currency': 'usd',
        'from': start_timestamp * 1000, # Multiply by 1000 for milliseconds
        'to': end_timestamp * 1000, # Multiply by 1000 for milliseconds
        'days': days_difference # Add the 'days' parameter
     }
    # Make the API request
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        prices = data.get('prices', [])
        return prices
    else:
        print(f"Failed to retrieve Bitcoin data. Status code: {response.status_code}")
        print(f"Error message: {response.text}")
    return None
    
# Set the date range
start_date = '2018-12-31'
end_date = '2023-12-29'
# Get historical Bitcoin data
bitcoin_prices = get_historical_bitcoin_data(start_date, end_date)
# Display the results in a simple table format

dates = [] 
prices_btc = []

if bitcoin_prices:
    print("{:<20} {:<15}".format("Date", "Price (USD)"))
    for timestamp, price in bitcoin_prices:
        date_str = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d')
        if date_str <= end_date:
            dates.append(date_str)
            prices_btc.append(price)
            #print("{:<20} {:<15.2f}".format(date_str, price))

# format = '%Y-%m-%d'
# for i in dates:
#     datetime.datetime.strptime(i, format).date()

df_bitcoin = pd.DataFrame(prices_btc, dates)
df_b

Date                 Price (USD)    


,0
2019-02-11,3631.444540
2019-02-12,3633.965047
2019-02-13,3610.062273
2019-02-14,3589.661830
2019-02-15,3601.229191


In [5]:
"""Antonio Section - Data
"""
csvpath = Path('FGRTX.csv')
fgrtx_df = pd.read_csv(csvpath)
# fgrtx_df.head()

#Sorting by "Date" 
# Sort data by `Date` in ascending order
fgrtx_df = fgrtx_df.sort_values("Date", ascending=True)
# fgrtx_df.head()

#Converted into datetime 
fgrtx_df.set_index(pd.to_datetime(fgrtx_df['Date'], infer_datetime_format=True), inplace=True)
# fgrtx_df.head()

#Drop the column
fgrtx_df.drop(columns=['Date','Open','High','Low','Close','Volume'], inplace=True)
fgrtx_df.head()

# Count nulls
fgrtx_df.isnull().sum()
# Drop nulls
fgrtx_df = fgrtx_df.dropna().copy()
fgrtx_df

#Check nulls
fgrtx_df.isnull().sum()

#Check datatypes
fgrtx_df.dtypes



/var/folders/jm/yl4jhp9j0z1b4xdws78jvgjc0000gn/T/ipykernel_89228/1384260641.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fgrtx_df.set_index(pd.to_datetime(fgrtx_df['Date'], infer_datetime_format=True), inplace=True)


Adj Close    float64
dtype: object

In [6]:
#Graph of the mutual fund: FGRTX 
fgrtx_df_price_data=fgrtx_df.hvplot(kind="line")
fgrtx_df_price_data

:Curve   [Date]   (Adj Close)

In [7]:
# Drop nulls
daily_returns_fgrtx = daily_returns_fgrtx.dropna().copy()
daily_returns_fgrtx



NameError: name 'daily_returns_fgrtx' is not defined

In [17]:
#Calling the CSV Path EFFR 
effr_df = pd.read_csv(Path('EFFR.csv'))

#Sorting the data by date, ascending 
effr_df_sorted = effr_df.sort_values("Effective Date", ascending=True)
effr_df_sorted.head()

effr_df_named = effr_df_sorted.rename(columns = 
                                      {"Effective Date":"Date"}
                                     )
# effr_df_named.head()

,Date,Rate Type,Rate (%),1st Percentile (%),25th Percentile (%),75th Percentile (%),99th Percentile (%),Volume ($Billions),Target Rate From (%),Target Rate To (%),Intra Day - Low (%),Intra Day - High (%),Standard Deviation (%),30-Day Average SOFR,90-Day Average SOFR,180-Day Average SOFR,SOFR Index,Revision Indicator (Y/N),Footnote ID
1255,01/02/2019,EFFR,2.40,2.36,2.40,2.41,2.60,56.0,2.25,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,01/02/2020,EFFR,1.55,1.52,1.54,1.55,1.61,74.0,1.50,1.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1254,01/03/2019,EFFR,2.40,2.34,2.40,2.41,2.60,62.0,2.25,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,01/03/2020,EFFR,1.55,1.50,1.54,1.55,1.61,79.0,1.50,1.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,01/03/2022,EFFR,0.08,0.06,0.08,0.08,0.15,74.0,0.00,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Traynor Dataframe


